<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# Processing TopsApp

## Introduction
In this notebook, we will run the various steps of processing with topsApp.py. 

topsApp.py is a pair-by-pair interferometric processor that takes as input two Sentinel-1 SAR acquisitions acquired in TOPS mode.At this time, topsApp only supports SLC data from Sentinel-1 A and B. Processing is supported across the Sentinel-1 constellation, i.e. data acquired from A and B can be combined**

Processing of TopsApp involves:

**Downloading Inputs**
  - Downloading SLCs : Both Reference and Secondary
  - Downlaoding DEMs : Based on supplied region of interest (min/max lat/lon)
  - Downloading Orbits : Based on SLC dates

**Processing with ISCE**
  - Creating ISCE input configuration files : topsApp.xml, reference.xml, secondary.xml
  - Steps of topsApp processing (in order):
     - startup
     - preprocess
     - computeBaselines**
     - verifyDEM
     - topo
     - subsetoverlaps
     - coarseoffsets
     - coarseresamp
     - overlapifg
     - prepesd
     - esd
     - rangecoreg
     - fineoffsets
     - fineresamp
     - ion
     - burstifg
     - mergebursts
     - filter
     - unwrap
     - unwrap2stage
     - geocode
     - denseoffsets
     - filteroffsets
     - geocodeoffsets
     
     The steps of topsApp depends on start and end parameter supplied while calling topsApp. For example, the following command will process from the first step (startup) till step 'geocode'
    ```
     /opt/isce2/isce/applications/topsApp.py --start=startup --end=geocode
    ```
    
    Also, some steps can be turned on or off by setting some properties in topsApp.xml. For example, 'unwrap' step will be ignored if'do_unwrap' property value is set to False
   


# Configurable Parameters

We can run this notebook to process topsApp with many different combinations of SLCs and region of interest as well as with different combination of topsApp properties like swaths, azimuth looks, range looks etc. In the followinng section we initizale all these tunable variables.

**Input Parameters**

- **min_lat, max_lat, min_loc, max_loc**: min/max latitude and longditude of Region of Interest.
    Example:
        min_lat = 31.9 # type: number
- **reference_slcs, secondary_slcs** : List of reference SLCs and secondary SLCs.
    Example:
        reference_slcs = ["S1B_IW_SLC__1SDV_20190628T014909_20190628T014936_016890_01FC87_55C8"]
        
**Tunable Parameters**
- **swaths** : array conntaining swath values to be considerate.
    Example:
        swaths = [3]
- **range_looks** : range looks value. Number.
    Example: 
        range_looks = 7        
- **azimuth_looks** : azimuth looks value. Number.
    Example:
        azimuth_looks = 3
- **do_unwrap** : True or False if unwrapping shoud be done or not.
    Example:
        do_unwrap = "True"
- **unwrapper_name** : Unwrapper name when do_unwrap is True.
    Example:
        unwrapper_name = "snaphu_mcf"
- **do_denseoffsets** : True/False for denseoffsets processing.
    Example:
        do_denseoffsets = "False"


In [ ]:

from typing import List

min_lat = 31.9 # type: number
max_lat = 33.94 # type: number
min_lon =  -118.74 # type: number
max_lon = -115.69 # type: number
        
reference_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190628T014909_20190628T014936_016890_01FC87_55C8"]
secondary_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190710T014909_20190710T014936_017065_0201B8_0252"]                     

    
sensor_name = "SENTINEL1"
swaths: List[int] = [1, 2, 3]
range_looks = 7
azimuth_looks = 3
do_unwrap = "False"
unwrapper_name = "snaphu_mcf"
do_denseoffsets = "False"


# Utility Functions : Plotting Functions

We use these functions to plot different figures to visualize the outputs. Matplotlib is used to generate the plots.

In [ ]:
from shutil import copyfile, move # Utilities for copying and moving files
from osgeo import gdal            # GDAL support for reading virtual files
import os                         # To create and remove directories
import matplotlib.pyplot as plt   # For plotting
import numpy as np                # Matrix calculations
import glob                       # Retrieving list of files
import boto3                      # For talking to s3 bucket
import rasterio as rio
from rasterio.plot import show, plotting_extent
from rasterio.merge import merge

# directory in which the notebook resides
if 'tutorial_home_dir' not in globals():
    tutorial_home_dir = os.getcwd()
print("Notebook directory: ", tutorial_home_dir)
# assumption is this is in the notebook_pges subdirectory of the repo
base_dir = os.path.abspath(os.path.join(tutorial_home_dir, '..'))

# directory for data downloads
output_dir = os.path.join(base_dir, 'notebook_output', 'topsApp')
slc_dir = os.path.join(output_dir, 'data', 'slcs')
orbit_dir = os.path.join(output_dir, 'data', 'orbits')
insar_dir = os.path.join(output_dir, 'insar')

# defining backup dirs in case of download issues on the local server
s3 = boto3.resource("s3")
data_backup_bucket = s3.Bucket("asf-jupyter-data")
data_backup_dir = "TOPS"

# generate all the folders in case they do not exist yet
os.makedirs(slc_dir, exist_ok=True)
os.makedirs(orbit_dir, exist_ok=True)
os.makedirs(insar_dir, exist_ok=True)

# Always start at the notebook directory    
os.chdir(tutorial_home_dir)

#Plot data using folio
def plot_wrapped_data_multiframe(frame_list):
    import os
    import folium
    from glob import glob
    import matplotlib.pyplot as plt
    import numpy as np
    import rasterio as rio
    from rasterio.plot import show, plotting_extent
    from rasterio.merge import merge
    from PIL import Image, ImageChops


    # plot wrapped IFGs individually
    flat_plots = []
    flat_bboxes = []
    for i, file in enumerate(frame_list):
        src = rio.open(file)
        fig, ax = plt.subplots(1, figsize=(18, 16))
        data = src.read(1)
        data[data==0] = np.nan
        show(np.angle(data), cmap='rainbow', vmin=-np.pi, vmax=np.pi, transform=src.transform, ax=ax)
        png_file = f'flat_{i}.png'
        fig.savefig(png_file, transparent=True)
        flat_plots.append(png_file)
        flat_bboxes.append(src.bounds)

def plot_wrapped_data_singleframe(filename='merged/filt_topophase.flat.geo'):
    import os
    import folium
    from glob import glob
    import matplotlib.pyplot as plt
    import numpy as np
    import rasterio as rio
    from rasterio.plot import show, plotting_extent
    from rasterio.merge import merge
    from PIL import Image, ImageChops


    src = rio.open(filename)

    fig, ax = plt.subplots(1, figsize=(18, 16))
    data = src.read(1)
    data[data==0] = np.nan
    show(np.angle(data), cmap='rainbow', vmin=-np.pi, vmax=np.pi, transform=src.transform, ax=ax)
    png_file = f'flat.png'
    fig.savefig(png_file, transparent=True)
    
# Utility to plot a 2D array
def plotdata(GDALfilename, band=1,
             title=None,colormap='gray',
             aspect=1, background=None,
             datamin=None, datamax=None,
             interpolation='nearest',
             nodata = None,
             draw_colorbar=True, colorbar_orientation="horizontal"):
    
    # Read the data into an array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    try:
        if nodata is not None:
            data[data == nodata] = np.nan
    except:
        pass
        
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+data.shape[1]*deltax
    lasty = firsty+data.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    if background is None:
        try:
            data[data==0]=np.nan
        except:
            pass
    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show()
    
    # clearing the data
    data = None

def plot_wrapped_multifiles(files, figsize=(20, 30)):
    files_to_mosaic = []
    for file in files: #glob("hello_world*/filt_topophase.flat.geo"):
        src = rio.open(file)
        files_to_mosaic.append(src)
    
    mosaic, out_trans = merge(files_to_mosaic, nodata=0)

    mosaic[mosaic==0] = np.nan
    fig, ax = plt.subplots(1, figsize=figsize)
    show(np.angle(mosaic[0]), cmap='rainbow', vmin=-np.pi, vmax=np.pi, ax=ax)

def plot_unwrapped_multifiles(files, figsize=(20, 30)):
    files_to_mosaic = []
    for file in files:
        src = rio.open(file)
        files_to_mosaic.append(src)
    
    mosaic, out_trans = merge(files_to_mosaic, nodata=0)

    mosaic[mosaic==0] = np.nan
    fig, ax = plt.subplots(1, figsize=figsize)
    show(mosaic[1], cmap='jet', vmin=-20, vmax=50, ax=ax)
    
# Utility to plot interferograms
def plotcomplexdata(GDALfilename,
                    title=None, aspect=1,
                    datamin=None, datamax=None,
                    interpolation='nearest',
                    draw_colorbar=None, colorbar_orientation="horizontal"):
    # Load the data into numpy array
    ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
    slc = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    ds = None
    
    # getting the min max of the axes
    firstx = transform[0]
    firsty = transform[3]
    deltay = transform[5]
    deltax = transform[1]
    lastx = firstx+slc.shape[1]*deltax
    lasty = firsty+slc.shape[0]*deltay
    ymin = np.min([lasty,firsty])
    ymax = np.max([lasty,firsty])
    xmin = np.min([lastx,firstx])
    xmax = np.max([lastx,firstx])

    # put all zero values to nan and do not plot nan
    try:
        slc[slc==0]=np.nan
    except:
        pass

    
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(slc), vmin = datamin, vmax=datamax,
                   cmap='gray', extent=[xmin,xmax,ymin,ymax],
                   interpolation=interpolation)
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(slc), cmap='rainbow',
                    vmin=-np.pi, vmax=np.pi,
                    extent=[xmin,xmax,ymin,ymax],
                    interpolation=interpolation)
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2, orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show()
    
    # clearing the data
    slc = None

# Utility to plot multiple similar arrays
def plotstackdata(GDALfilename_wildcard, band=1,
                  title=None, colormap='gray',
                  aspect=1, datamin=None, datamax=None,
                  interpolation='nearest',
                  draw_colorbar=True, colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    
    # initialize empty numpy array
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(band).ReadAsArray()   
        ds = None
        
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass            
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111)
    cax = ax.imshow(data, vmin = datamin, vmax=datamax,
                    cmap=colormap, interpolation=interpolation)
    ax.set_title(title)
    if draw_colorbar is not None:
        cbar = fig.colorbar(cax,orientation=colorbar_orientation)
    ax.set_aspect(aspect)    
    plt.show() 

    # clearing the data
    data = None

# Utility to plot multiple simple complex arrays
def plotstackcomplexdata(GDALfilename_wildcard,
                         title=None, aspect=1,
                         datamin=None, datamax=None,
                         interpolation='nearest',
                         draw_colorbar=True, colorbar_orientation="horizontal"):
    # get a list of all files matching the filename wildcard criteria
    GDALfilenames = glob.glob(GDALfilename_wildcard)
    print(GDALfilenames)
    # initialize empty numpy array
    data = None
    for GDALfilename in GDALfilenames:
        ds = gdal.Open(GDALfilename, gdal.GA_ReadOnly)
        data_temp = ds.GetRasterBand(1).ReadAsArray()
        ds = None
        
        if data is None:
            data = data_temp
        else:
            data = np.vstack((data,data_temp))

    # put all zero values to nan and do not plot nan
    try:
        data[data==0]=np.nan
    except:
        pass              
            
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(1,2,1)
    cax1=ax.imshow(np.abs(data), vmin=datamin, vmax=datamax,
                   cmap='gray', interpolation='nearest')
    ax.set_title(title + " (amplitude)")
    if draw_colorbar is not None:
        cbar1 = fig.colorbar(cax1,orientation=colorbar_orientation)
    ax.set_aspect(aspect)

    ax = fig.add_subplot(1,2,2)
    cax2 =ax.imshow(np.angle(data), cmap='rainbow',
                            interpolation='nearest')
    ax.set_title(title + " (phase [rad])")
    if draw_colorbar is not None:
        cbar2 = fig.colorbar(cax2,orientation=colorbar_orientation)
    ax.set_aspect(aspect)
    plt.show() 
    
    # clearing the data
    data = None

## Other Utility Functions 
Functions related to perform downloading of inputs, such as SLCs, dems, orbits etc and generating the config files dynamically.

In [ ]:
import os
import json
from math import floor, ceil
import json
import re
import osaka
import osaka.main
from builtins import str
import os, sys, re, json, logging, traceback, requests, argparse
from datetime import datetime
from pprint import pformat
from requests.packages.urllib3.exceptions import (InsecureRequestWarning,
                                                  InsecurePlatformWarning)
import isce
from iscesys.Component.ProductManager import ProductManager as PM

try: from html.parser import HTMLParser
except: from html.parser import HTMLParser
    
log_format = "[%(asctime)s: %(levelname)s/%(funcName)s] %(message)s"
logging.basicConfig(format=log_format, level=logging.INFO)
logger = logging.getLogger('create_ifg')

        
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecurePlatformWarning)
PROCESSING_START=datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

PGE_BASE=os.getcwd()
ISCE_HOME="/opt/isce2/isce"

SLC_RE = re.compile(r'(?P<mission>S1\w)_IW_SLC__.*?' +
                    r'_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})' +
                    r'T(?P<start_hour>\d{2})(?P<start_min>\d{2})(?P<start_sec>\d{2})' +
                    r'_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})' +
                    r'T(?P<end_hour>\d{2})(?P<end_min>\d{2})(?P<end_sec>\d{2})_.*$')


QC_SERVER = 'https://qc.sentinel1.eo.esa.int/'
DATA_SERVER = 'http://aux.sentinel1.eo.esa.int/'

ORBITMAP = [('precise','aux_poeorb', 100),
            ('restituted','aux_resorb', 100)]

OPER_RE = re.compile(r'S1\w_OPER_AUX_(?P<type>\w+)_OPOD_(?P<yr>\d{4})(?P<mo>\d{2})(?P<dy>\d{2})')

wd = os.getcwd()

if isinstance(reference_slcs, str):
    reference_slcs = json.loads(reference_slcs)
if isinstance(secondary_slcs, str):
    secondary_slcs = json.loads(secondary_slcs)
    
if isinstance(swaths, str):
    swaths = json.loads(swaths)
    
localize_slcs = reference_slcs + secondary_slcs
tops_properties = {}
tops_properties["swaths"] = swaths
tops_properties["range looks"] = range_looks
tops_properties["azimuth looks"] = azimuth_looks
tops_properties["do unwrap"] = do_unwrap
tops_properties["unwrapper name"] = unwrapper_name
tops_properties["do denseoffsets"] = do_denseoffsets
tops_properties["region of interest"] ="[{}, {}, {}, {}]".format(min_lat, max_lat, min_lon, max_lon)
wgs84_file = ''

class MyHTMLParser(HTMLParser):

    def __init__(self):
        HTMLParser.__init__(self)
        self.fileList = []
        self.pages = 0
        self.in_td = False
        self.in_a = False
        self.in_ul = False

    def handle_starttag(self, tag, attrs):
        if tag == 'td':
            self.in_td = True
        elif tag == 'a' and self.in_td:
            self.in_a = True
        elif tag == 'ul':
            for k,v in attrs:
                if k == 'class' and v.startswith('pagination'):
                    self.in_ul = True
        elif tag == 'li' and self.in_ul:
            self.pages += 1

    def handle_data(self,data):
        if self.in_td and self.in_a:
            if OPER_RE.search(data):
                self.fileList.append(data.strip())

    def handle_endtag(self, tag):
        if tag == 'td':
            self.in_td = False
            self.in_a = False
        elif tag == 'a' and self.in_td:
            self.in_a = False
        elif tag == 'ul' and self.in_ul:
            self.in_ul = False
        elif tag == 'html':
            if self.pages == 0:
                self.pages = 1
            else:
                # decrement page back and page forward list items
                self.pages -= 2

def session_get(session, url):
    return session.get(url, verify=False)

def get_download_orbit_dict(download_orbit_dict, slc_date, mission_type):
    

    logger.info("slc_date : {}".format(slc_date))
    url = "https://qc.sentinel1.eo.esa.int/aux_poeorb/?validity_start={}&sentinel1__mission={}".format(slc_date, mission_type)
    session = requests.Session()
    r = session_get(session, url)
    r.raise_for_status()
    parser = MyHTMLParser()
    parser.feed(r.text)

    for res in parser.fileList:
        #id = "%s-%s" % (os.path.splitext(res)[0], dataset_version)
        match = OPER_RE.search(res)
        if not match:
            raise RuntimeError("Failed to parse orbit: {}".format(res))
        download_orbit_dict[res] = os.path.join(DATA_SERVER, "/".join(match.groups()), "{}.EOF".format(res))
        #yield id, results[id]
        
    #logger.info(results)
    
    return download_orbit_dict

def get_orbit_files(localize_slcs):
    from datetime import datetime, timedelta
    import json
    import os
    import osaka
    import osaka.main
    
    orbit_dict = {}

    orbit_dates = []
    
    for slc in localize_slcs:
        match = SLC_RE.search(slc)
        if not match:
            raise RuntimeError("Failed to recognize SLC ID %s." %slc)
        mission = match.group('mission')
        day_dt_str = "{}-{}-{}".format(match.group('start_year'), 
                                       match.group('start_month'), match.group('start_day'))
        
        day_dt = datetime.strptime(day_dt_str, '%Y-%m-%d') - timedelta(days=1)
        
        day_dt_str = day_dt.strftime('%Y-%m-%d')
        
        logger.info("day_dt_str : {}".format(day_dt_str))
        if day_dt_str not in orbit_dates:
            orbit_dates.append(day_dt_str)
            orbit_dict = get_download_orbit_dict(orbit_dict, day_dt_str, mission)
            directory = orbit_dir
            if not os.path.exists(directory):
                os.makedirs(directory)
            #directory = os.path.join(wd, "orbits")
            for k, v in orbit_dict.items():
                osaka.main.get(v, directory)
         
    logger.info("orbit_dict : %s " %json.dumps(orbit_dict, indent=4))

def get_area(coords):
    '''get area of enclosed coordinates- determines clockwise or counterclockwise order'''
    from past.utils import old_div
    
    n = len(coords) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += coords[i][1] * coords[j][0]
        area -= coords[j][1] * coords[i][0]
    #area = abs(area) / 2.0
    return old_div(area, 2)

def download_slc(slc_id, path):
    url = "https://datapool.asf.alaska.edu/SLC/SA/{}.zip".format(slc_id)
    logger.info("Downloading {} : {}".format(slc_id, url))
    
    if not os.path.exists(path):
        os.makedirs(path)
    osaka.main.get(url, path)

def run_cmd_output(cmd):
    from subprocess import check_output, CalledProcessError
    cmd_line = " ".join(cmd)
    logger.info("Calling: {}".format(cmd_line))
    output = check_output(cmd_line, shell=True)
    return output

def run_cmd(cmd):
    !source /opt/isce2/isce_env.sh
    !export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
    !export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}

    import subprocess
    from subprocess import check_call, CalledProcessError
    import sys
    cmd_line = " ".join(cmd)
    logger.info("Calling : {}".format(cmd_line))
    p = subprocess.Popen(cmd_line, shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    while True: 
        line = p.stdout.readline()
        if not line:
            break
        logger.info(line.strip())
        sys.stdout.flush()
        
def download_dem(min_lat, max_lat, min_lon, max_lon):
    from math import floor, ceil
    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)
    
    dem_cmd = [
        "{}/applications/dem.py".format(ISCE_HOME), "-a",
        "stitch", "-b", "{} {} {} {}".format(min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi),
        "-r", "-s", "1", "-f", "-c", "|", "tee", "dem.txt"
        #"-n", dem_user, "-w", dem_pass,"-u", dem_url
    ]
    run_cmd(dem_cmd)
        
def download_slcs(localize_slcs, path):
    for slc in localize_slcs:
        download_slc(slc, path) 
        
def get_start_end_times(localize_slcs):
    #"S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768"
    import re
    import datetime
    
    start_times = []
    end_times = []
    SLC_RE1 = re.compile(r'(?P<mission>S1\w)_IW_SLC__.*?_(?P<start_time>\d{8}T\d{6})_(?P<end_time>\d{8}T\d{6})_.*$')
    for slc in localize_slcs:
        match = SLC_RE1.search(slc)
        if not match:
            raise RuntimeError("Failed to recognize SLC ID %s." %slc)
        start_time_str = "{}".format(match.group('start_time'))
        start_times.append(datetime.datetime.strptime(start_time_str, '%Y%m%dT%H%M%S'))
        end_time_str = "{}".format(match.group('end_time'))
        end_times.append(datetime.datetime.strptime(end_time_str, '%Y%m%dT%H%M%S'))
    return sorted(start_times)[0], sorted(end_times)[-1]
                                     
        
def xml2string(xmlroot, encoding="UTF-8", method="xml", indent="\t", newl="\n"):
    from xml.dom import minidom
    import xml.etree.cElementTree as ET
    
    xmlstring = minidom.parseString(ET.tostring(xmlroot, encoding=encoding, method=method))\
        .toprettyxml(newl=newl, indent=indent)
    return xmlstring

def create_xml(xml_file, doc_type, slcs):
    from xml.dom import minidom 
    import xml.etree.cElementTree as ET
    import os 
   
    slc_list = []
    for slc in slcs:
        slc_list.append(os.path.join('../data/slcs/', "{}.zip".format(slc)))

    

    comp_elem = ET.Element('component')
    comp_elem.set("name", doc_type)

    prop_elem = ET.SubElement(comp_elem, 'property') 
    prop_elem.set('name', 'orbit directory')
    prop_elem.text='../data/orbits'

    prop_elem = ET.SubElement(comp_elem, 'property') 
    prop_elem.set('name', 'output directory')
    prop_elem.text=doc_type
    
    prop_elem = ET.SubElement(comp_elem, 'property') 
    prop_elem.set('name', 'safe')
    prop_elem.text=str(slc_list)


    xml_str = xml2string(comp_elem) 

    with open(xml_file, 'w') as fw:
        fw.write(xml_str)
        
def create_dataset_json(id, version, met_file, ds_file):
    """Write dataset json."""


    # get metadata
    with open(met_file) as f:
        md = json.load(f)

    # build dataset
    ds = {
        'creation_timestamp': "%sZ" % datetime.utcnow().isoformat(),
        'version': version,
        'label': id
    }

    try:
        
        
        
        logger.info("create_dataset_json : met['bbox']: %s" %md['bbox'])
        
        coordinates = [
                    [
                      [ md['bbox'][0][1], md['bbox'][0][0] ],
                      [ md['bbox'][3][1], md['bbox'][3][0] ],
                      [ md['bbox'][2][1], md['bbox'][2][0] ],
                      [ md['bbox'][1][1], md['bbox'][1][0] ],
                      [ md['bbox'][0][1], md['bbox'][0][0] ]
                    ] 
                  ]
        
     
        #coordinates = md['union_geojson']['coordinates']
    
        cord_area = get_area(coordinates[0])
        if not cord_area>0:
            logger.info("creating dataset json. coordinates are not clockwise, reversing it")
            coordinates = [coordinates[0][::-1]]
            logger.info(coordinates)
            cord_area = get_area(coordinates[0])
            if not cord_area>0:
                logger.info("creating dataset json. coordinates are STILL NOT  clockwise")
        else:
            logger.info("creating dataset json. coordinates are already clockwise")

        ds['location'] =  {'type': 'Polygon', 'coordinates': coordinates}
        logger.info("create_dataset_json location : %s" %ds['location'])

    except Exception as err:
        logger.info("create_dataset_json: Exception : ")
        logger.info(str(err))
        logger.info("Traceback: {}".format(traceback.format_exc()))


    # set earliest sensing start to starttime and latest sensing stop to endtime
    if isinstance(md['sensing_start'], str):
        ds['starttime'] = md['sensing_start']
    else:
        md['sensing_start'].sort()
        ds['starttime'] = md['sensing_start'][0]

    if isinstance(md['sensing_stop'], str):
        ds['endtime'] = md['sensing_stop']
    else:
        md['sensing_stop'].sort()
        ds['endtime'] = md['sensing_stop'][-1]

    # write out dataset json
    with open(ds_file, 'w') as f:
        json.dump(ds, f, indent=2)
        

def get_tops_subswath_xml(masterdir):
    ''' 
        Find all available IW[1-3].xml files
    '''

    logger.info("get_tops_subswath_xml from : %s" %masterdir)

    masterdir = os.path.abspath(masterdir)
    IWs = glob(os.path.join(masterdir,'IW*.xml'))
    if len(IWs)<1:
        raise Exception("Could not find a IW*.xml file in " + masterdir)

    return IWs

def read_isce_product(xmlfile):
    logger.info("read_isce_product: %s" %xmlfile)

    # check if the file does exist
    check_file_exist(xmlfile)

    # loading the xml file with isce
    pm = PM()
    pm.configure()
    obj = pm.loadProduct(xmlfile)
    return obj

def check_file_exist(infile):
    logger.info("check_file_exist : %s" %infile)
    if not os.path.isfile(infile):
        raise Exception(infile + " does not exist")
    else:
        logger.info("%s Exists" %infile)


def get_tops_metadata(masterdir):

    logger.info("get_tops_metadata from : %s" %masterdir)
    # get a list of avialble xml files for IW*.xml
    IWs = get_tops_subswath_xml(masterdir)
    # append all swaths togheter
    frames=[]
    for IW  in IWs:
        logger.info("get_tops_metadata processing : %s" %IW)
        obj = read_isce_product(IW)
        frames.append(obj)

    output={}
    dt = min(frame.sensingStart for frame in frames)
    output['sensingStart'] =  dt.isoformat('T') + 'Z'
    logger.info(dt)
    dt = max(frame.sensingStop for frame in frames)
    output['sensingStop'] = dt.isoformat('T') + 'Z'
    logger.info(dt)
    return output
        
def extract_slc_data(slc_dir, slcs):
    from zipfile import ZipFile
    os.chdir(slc_dir)
    for slc in slcs:
        i = "{}.zip".format(slc)
        with ZipFile(i, 'r') as zf:
            zf.extractall()
            
def create_product(insar_dir, tops_properties):
            
    from datetime import datetime
    from glob import glob
    import shutil

    os.chdir(insar_dir)
    print(insar_dir)

    '''
    output = get_tops_metadata('fine_interferogram')
    sensing_start= output['sensingStart']
    sensing_stop = output['sensingStop']
    logger.info("sensing_start : %s" %sensing_start)
    logger.info("sensing_stop : %s" %sensing_stop)
    '''


    now=datetime.utcnow().strftime("%Y%m%dT%H%M%S")
    dataset_name = "hello_world-product-{}-{}".format(now, "TopsApp")
    start_time, end_time = get_start_end_times(localize_slcs)
    print("{} : {}".format(start_time, end_time))
    #print(dataset_name)

    prod_dir = os.path.join(insar_dir, dataset_name)
    os.mkdir(prod_dir)

    met_file = os.path.join(dataset_name, "{}.met.json".format(dataset_name))
    dataset_file = os.path.join(dataset_name, "{}.dataset.json".format(dataset_name))

    met = tops_properties
    met["reference_slc"]=reference_slcs
    met["secondary_slcs"] = secondary_slcs
    met['sensing_start'] = start_time.isoformat('T') + 'Z'
    met['sensing_stop'] = end_time.isoformat('T') + 'Z'
    met['start_time'] = start_time.isoformat('T') + 'Z'
    met['end_time'] = end_time.isoformat('T') + 'Z'

    bbox = [[min_lat, min_lon], [min_lat, max_lon], [max_lat, max_lon], [max_lat, min_lon]]
    met['bbox'] =  bbox 
    version = "v1.0"

    # generate dataset JSON
    with open(met_file, 'w') as f: json.dump(met, f, indent=2)
    
    # generate dataset JSON
    ds_file = os.path.join(prod_dir, "{}.dataset.json".format(dataset_name))
    create_dataset_json(dataset_name, version, met_file, ds_file)

    merged_dir = os.path.join(insar_dir, "merged")
    for name in glob("{}/*".format(merged_dir)):
    
        input_path = os.path.join(merged_dir, name)
        print(input_path)
        if os.path.isfile(input_path):
            #print("Copying {} to {}".format(input_path,  prod_dir ))
            shutil.copy(input_path,  prod_dir)
    return prod_dir
            
def create_topsApp_xml():
    from xml.dom import minidom 
    import xml.etree.cElementTree as ET
    import os 

    supported_docs = os.path.join(tutorial_home_dir, 'support_docs', 'insar')
    os.makedirs(supported_docs, exist_ok=True)


    tops_xml_file = os.path.join(tutorial_home_dir, "support_docs/insar/topsApp.xml")

    root = ET.Element("topsApp")

    comp_elem = ET.SubElement(root, 'component')
    comp_elem.set("name", 'topsinsar')

    prop_elem = ET.SubElement(comp_elem, 'property') 
    prop_elem.set('name', 'Sensor name')
    prop_elem.text=sensor_name

    ref_elem = ET.SubElement(comp_elem, 'component')
    ref_elem.set("name", 'reference')

    ref_cat_elem = ET.SubElement(ref_elem, 'catalog')
    ref_cat_elem.text = "reference.xml"

    sec_elem = ET.SubElement(comp_elem, 'component')
    sec_elem.set("name", 'secondary')

    ref_cat_elem = ET.SubElement(sec_elem, 'catalog')
    ref_cat_elem.text = "secondary.xml"

    for k, v in tops_properties.items():
        prop_elem = ET.SubElement(comp_elem, 'property')
        prop_elem.set('name', k)
        prop_elem.text = str(v)

    prop_elem = ET.SubElement(comp_elem, 'property') 
    prop_elem.set('name', 'demfilename')
    prop_elem.text=wgs84_file
   
    xml_str = xml2string(root) 

    with open(tops_xml_file, 'w') as fw:
        #fw.write(r'<?xml version="1.0" encoding="UTF-8"?>\n')
        fw.write(xml_str)

## 1. Collecting Input Datasets
 - Downnloading **SLCs**
 - Downloading **Orbits**
 - Downloading **Dems**
 

### 1.1 SLC Download

TOPS SLC product files delivered from ESA are zip archives. When unpacked the zip extension will be replaced by SAFE. The products are therefore also frequently called SAFE files. topsApp.py can read the data from either a zip file or a SAFE file. To limit disk usage, it is recommended to not unzip the individual files.

The zip or SAFE filenames provide information on the product type, the polarization, and the start and stop acquisition time. For example: S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip
- Type = slc
- Polarization = Dual polarization
- Date = 20200511
- UTC time of acquisition = ~13:51
- Sensing start for the acquisition was 20200511 at 13:51:17


In [ ]:
download_slcs(localize_slcs, slc_dir)

! ls -lh {slc_dir}

### 1.2 Orbits Download

In addition to the **SAFE files**, **orbit files** and the **auxiliary instrument files** are required for ISCE processing. Both the orbit and instrument files are provided by ESA and can be downloaded at: https://qc.sentinel1.eo.esa.int/.

In [ ]:
from shutil import move

os.chdir(tutorial_home_dir)
print(tutorial_home_dir)

get_orbit_files(localize_slcs)
# Move the orbits to orbit folder
orb_files = glob.glob("*.EOF")
for orb in orb_files:
    move(orb, os.path.join(orbit_dir, orb))
!ls {orbit_dir}

### 1.3 Dems Download

Dems over the region of intetrest is downloaded.

In [ ]:
import os

os.chdir(insar_dir)
download_dem(min_lat, max_lat, min_lon, max_lon)

wgs84_file =''
if os.path.exists("dem.txt"):
    cmd = ["awk", "'/wgs84/ {print $NF;exit}'", "dem.txt"]
    WGS84 = run_cmd_output(cmd).decode("utf-8").strip()
    wgs84_file = os.path.join(".", WGS84)
print(wgs84_file)

### 1.4 AUX_CAL file download ####

The following calibration auxliary (AUX_CAL) file is used for **antenna pattern correction** to compensate the range phase offset of SAFE products with **IPF verison 002.36** (mainly for images acquired before March 2015). If all your SAFE products are from another IPF version, then no AUX files are needed. Check [ESA document](https://earth.esa.int/documents/247904/1653440/Sentinel-1-IPF_EAP_Phase_correction) for details. 

Run the command below to download the AUX_CAL file once and store it somewhere (_i.e._ ~/aux/aux_cal) so that you can use it all the time, for `stackSentinel.py -a` or `auxiliary data directory` in `topsApp.py`.

```
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ

In [ ]:
%%bash
cd $insar_dir
mkdir -p ./AuxDir
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ --directory ./AuxDir
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ

## 2 topsApp Connfiguration Files


For topsApp processing, we use *topsApp.xml*, *reference.xml* and *secondary.xml*  as config files to send input data information to isce.

### 2.1 topsApp.xml

Example:
```xml
<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
  <component name="topsinsar">
    <property name="Sensor name">SENTINEL1</property>
    <component name="reference">
        <catalog>reference.xml</catalog>
    </component>
    <component name="secondary">
        <catalog>secondary.xml</catalog>
    </component>
    <property name="swaths">[3]</property>
    <property name="range looks">7</property>
    <property name="azimuth looks">3</property>
    <property name="region of interest">[37.98, 38.33, -118.21, -117.68]</property>
    <property name="do unwrap">True</property>
    <property name="unwrapper name">snaphu_mcf</property>
    <property name="do denseoffsets">True</property>
    <property name="demfilename">path_to_your_dem</property>
    <property name="geocode demfilename">path_to_your_dem</property>
    <!--property name="geocode list">['merged/phsig.cor', 'merged/filt_topophase.unw', 'merged/los.rdr', 'merged/topophase.flat', 'merged/filt_topophase.flat','merged/topophase.cor','merged/filt_topophase.unw.conncomp']</property>-->
  </component>
</topsApp>
```

- The reference and secondary components refer  to their own *.xml* files 
- The **swaths** property controls the number of swaths to be processed. 
- **range looks** and **azimuth looks**: The range resolution for sentinel varies from near to far range, but is roughly 5m, while the azimuth resolution is approximately 15m, leading to a multi-looked product that will be approximately 35m by 45m.
- By specifying the **region of interest** as [S, N, W, E] to only capture the extent, topsApp.py will only extract those bursts from subswaths needed to cover the earthquake.
- By default, topsApp can download a DEM on the fly. By including **demFilename** a local DEM can be specified as input for the processing.
- By default, the geocoding in topsApp.py is performed at the same sampling as processing DEM. However, a different DEM *to be used specifically for geocoding* can be specified using the **geocode demfilename** property. This is used for the case when data has been multilooked to order of 100m or greater and when geocoding to 30m is an overkill.
- By default, no unwrapping is done. In order to turn it on, set the property **do unwrap** to *True*.
- In case unwrapping is requested, the default unwrapping strategy to be applied is the *icu* unwrapping method. For this tutorial, we will use *snaphu_mcf*.
- Lastly, we request topsApp.py to run the dense-offsets using the **do denseoffsets** property. By enabling this, topsApp.py will estimate the range and azimuth offsets on the amplitude of the reference and secondary SLC files.

In [ ]:
create_topsApp_xml()

### 2.2 reference.xml and secondary.xml

Example:
``` xml
<component name="reference">
    <property name="orbit directory">../data/orbits</property>
    <property name="output directory">reference</property>
    <property name="safe">['../data/slcs/S1A_IW_SLC__1SDV_20200511T135117_20200511T135144_032518_03C421_7768.zip']</property>
</component>
```
- The value associated with the reference **safe** property corresponds to a list of SAFE files that are to be mosaiced when generating the interferogram. 
- The **orbit directory** points  to the directory where we have stored the POEORB (precise) orbits for this example.

In [ ]:
xml_file = os.path.join(tutorial_home_dir, "support_docs/insar/reference.xml")
create_xml(xml_file, 'reference', reference_slcs)

xml_file = os.path.join(tutorial_home_dir, "support_docs/insar/secondary.xml")
create_xml(xml_file, 'secondary', secondary_slcs)

### 2.3 Moving input XML Files in Appropriate Directory

Moving the files in insar directory.

In [ ]:
## Template xml for this tutorial
topsAppXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/topsApp.xml')  
refXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/reference.xml')  
secXml_original =  os.path.join(tutorial_home_dir,'support_docs/insar/secondary.xml')  


## Check if the topsApp.xml file already exists, if not copy the example for the excerisize
if not os.path.isfile(os.path.join(insar_dir,'topsApp.xml')):
    copyfile(topsAppXml_original, os.path.join(insar_dir, 'topsApp.xml'))
else:
    print(os.path.join(insar_dir,'topsApp.xml') + " already exist, will not overwrite")
    
if not os.path.isfile(os.path.join(insar_dir, 'reference.xml')):
    copyfile(refXml_original, os.path.join(insar_dir,'reference.xml'))
else:
    print(os.path.join(insar_dir,'reference.xml') + " already exist, will not overwrite")
    
if not os.path.isfile(os.path.join(insar_dir, 'secondary.xml')):
    copyfile(secXml_original,os.path.join(insar_dir, 'secondary.xml'))
else:
    print(os.path.join(insar_dir,'secondary.xml') + " already exist, will not overwrite")

## 3. topsApp.py processing steps

The topsApp.py workflow can be called with a single command-line call to topsApp.py; by default it will run all the required processing steps with inputs pulled from the topsApp.xml file. Although this is an attractive feature, it is recommended to run topsApp.py with “steps” enabled. This will allow you to re-start the processing from a given processing step. If “steps” are not used, users must restart processing from the beginning of the workflow after fixing any downstream issues with the processing.


**Steps of topsApp processing (in order)**:
 - startup
 - preproces
 - computeBaselines
 - verifyDEM
 - topo
 - subsetoverlaps
 - coarseoffsets
 - coarseresamp
 - overlapifg
 - prepesd
 - esd
 - rangecoreg
 - fineoffsets
 - fineresamp
 - ion
 - burstifg
 - mergebursts
 - filter
 - unwrap
 - unwrap2stage
 - geocode
 - denseoffsets
 - filteroffsets
 - geocodeoffsets
 


In [ ]:
!/opt/isce2/isce/applications/topsApp.py --help --steps

### 3.1 START THE PROCESSING
We will do processing from "startup" to "geocode" in one step in working directory (insar)

In [ ]:
os.chdir(insar_dir)
!/opt/isce2/isce/applications/topsApp.py --start=startup --end=geocode

### 3.2 PLOT  INTERFEROGRAM

Plotting filt_topophase.unw.geo with metaplotlib

In [ ]:
import os
os.chdir(insar_dir)

file = os.path.join(insar_dir, 'merged', 'filt_topophase.flat.geo')
print(file)

'''
plotdata('merged/filt_topophase.unw.geo', band=2,
         title="UNW GEO FILT IFG [rad] ", colormap='jet',
         colorbar_orientation="vertical", datamin=-20, datamax=50)
'''
try:
    plot_wrapped_data_multiframe([file])
except Exception as e:
    print(e)


### 3.3 Generate the Product

In [ ]:
prod_name = create_product(insar_dir, tops_properties)

print(prod_name)

# Relevant references:
- Heresh Fattahi, Piyush Agram, and Mark Simons, *Precise coregistration of Sentinel-1A TOPS data*, https://files.scec.org/s3fs-public/0129_1400_1530_Fattahi.pdf
- Fattahi, H., Agram, P. and Simons, M., 2016. A network-based enhanced spectral diversity approach for TOPS time-series analysis. IEEE Transactions on Geoscience and Remote Sensing, 55(2), pp.777-786. https://core.ac.uk/reader/77927508
- ESA, *Sentinel-1 and TOPS overview*, https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar
- Nestor Yague-Martinez, Pau Prats-Iraola, Fernando Rodriguez Gonzalez,Ramon Brcic, Robert Shau, Dirk Geudtner, Michael Eineder, and Richard Bamler, *Interferometric Processing of Sentinel-1 TOPS Data*, IEEE, doi:10.1109/TGRS.2015.2497902, https://ieeexplore.ieee.org/document/7390052/
- Liang, C., Agram, P., Simons, M. and Fielding, E.J., 2019. Ionospheric correction of insar time series analysis of c-band sentinel-1 tops data. IEEE Transactions on Geoscience and Remote Sensing, 57(9), pp.6755-6773. 

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.4 and above</font>